In [32]:
import pandas as pd

In [33]:
df = pd.read_csv("dados/train.csv")

In [34]:
df1 = df.copy()

In [35]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Vehicle_conditio

In [36]:
df1.isnull().sum()

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Time_Order_picked              0
Weatherconditions              0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken(min)                0
dtype: int64

## Limpeza de dados

In [37]:
#Removendo espaço nas strings
def retira_espaco(df):
    columns = dict(df.dtypes == "object")
    strings = []
    for col in columns:
        if columns[col] == True:
            strings.append(col)
    for str in strings:
        df[str] = df[str].str.strip()
        
    return df

def retorna_lista_nan(df):
    columns = dict(df.dtypes == "object")
    strings = []
    for col in columns:
        if columns[col] == True:
            strings.append(col)

    lista = []
    for str in strings:
        if df[str].str.contains("NaN").sum() > 0:
            lista.append(str)
    return lista

In [38]:
df1 = retira_espaco(df1)

In [39]:
lista_nan = retorna_lista_nan(df1)
lista_nan

['Delivery_person_Age',
 'Delivery_person_Ratings',
 'Time_Orderd',
 'Weatherconditions',
 'Road_traffic_density',
 'multiple_deliveries',
 'Festival',
 'City']

In [40]:
# #Retirando valores NaN
df1 = df1[~df1["Delivery_person_Age"].str.contains("NaN")]
df1 = df1[~df1["Delivery_person_Ratings"].str.contains("NaN")]
df1 = df1[~df1["Time_Orderd"].str.contains("NaN")]
df1 = df1[~df1["Weatherconditions"].str.contains("NaN")]
df1 = df1[~df1["Road_traffic_density"].str.contains("NaN")]
df1 = df1[~df1["multiple_deliveries"].str.contains("NaN")]
df1 = df1[~df1["Festival"].str.contains("NaN")]
df1 = df1[~df1["City"].str.contains("NaN")]
df1 = df1.reset_index(drop=True)

In [41]:
lista_nan = retorna_lista_nan(df1)
lista_nan

[]

In [42]:
df1["Delivery_person_Age"] = df1["Delivery_person_Age"].astype( int )
df1["Delivery_person_Ratings"] = df1["Delivery_person_Ratings"].astype( float )
df1["Order_Date"] = pd.to_datetime(df1["Order_Date"], format="%d-%m-%Y")
df1["multiple_deliveries"] = df1["multiple_deliveries"].astype( int )
df1["Time_taken(min)"] = df1["Time_taken(min)"].replace(to_replace="\(min\) ", value="", regex=True).astype(int)

# 1 - Visão: Empresa

#### 1.1 - Quantidade de pedidos por dia (Gráfico de barra)

In [43]:
import plotly.express as px

In [44]:
pedidos_dia_df = df.groupby("Order_Date").agg({"ID":"count"}).reset_index().rename(columns={"ID":"Qtde_Pedidos"})
pedidos_dia_df["Order_Date"] = pd.to_datetime(pedidos_dia_df["Order_Date"], format="%d-%m-%Y")


px.bar(data_frame=pedidos_dia_df, x="Order_Date", y="Qtde_Pedidos")

1.2 - Quantidade de pedidos por semana (gráfico de linha)

In [45]:
#Criando coluna de semana
df1["week_of_year"] = df1["Order_Date"].dt.strftime("%U")
pedidos_semana_df = df1.groupby("week_of_year").agg({"ID":"count"}).reset_index().rename(columns={"ID":"Qtde_Pedido"})

px.line(data_frame=pedidos_semana_df, x="week_of_year", y="Qtde_Pedido")

#### 1.3 - Distribuição dos pedidos por tipo de tráfego (Gráfico de pizza)

In [46]:
pedidos_trafego_df = df1.groupby("Road_traffic_density").agg({"ID":"count"}).reset_index().rename(columns={"ID":"Qtde_Pedidos"})
pedidos_trafego_df["entregas_perc"] = pedidos_trafego_df["Qtde_Pedidos"]/pedidos_trafego_df["Qtde_Pedidos"].sum()

px.pie(data_frame=pedidos_trafego_df, names="Road_traffic_density", values="entregas_perc")

#### 1.4 - Comparação do volume de pedidos por cidade e tipo de tráfego

In [47]:
cidade_trafego_df = df1.groupby(["City", "Road_traffic_density"]).agg({"ID":"count"}).reset_index().rename(columns={"ID":"Qtde_Pedidos"})

px.scatter(data_frame=cidade_trafego_df, x="City", y="Road_traffic_density", size="Qtde_Pedidos", color="City")

#### 1.5 - Quantidade de pedidos por entregador por semana 

In [48]:
#Quantidade de pedios por semana/número únido de entregadores por semana
temp_pedido_por_semana = df1.groupby("week_of_year").agg({"ID":"count"}).reset_index().rename(columns={"ID":"Qtde_Pedidos"})
temp_entregadores_por_semana = df1.groupby("week_of_year").agg({"Delivery_person_ID":"nunique"}).reset_index().rename(columns={"Delivery_person_ID":"Entregadores_Unicos"})

pedidos_entregadores_semana = temp_pedido_por_semana.merge(temp_entregadores_por_semana, on="week_of_year", how="inner")
pedidos_entregadores_semana["order_by_delivery"] = pedidos_entregadores_semana["Qtde_Pedidos"]/pedidos_entregadores_semana["Entregadores_Unicos"]

px.line(data_frame=pedidos_entregadores_semana, x="week_of_year", y="order_by_delivery")


#### 1.6 - A localização central de cada cidade por tipo de tráfego 

In [49]:
#Biblioteca para mapas
import folium

In [50]:
loc_central_df = df1.groupby(["City", "Road_traffic_density"]).agg({"Delivery_location_latitude":"median", "Delivery_location_longitude":"median"}).reset_index()

map = folium.Map()

In [51]:
for index, location_info in loc_central_df.iterrows():
    folium.Marker(location=[location_info["Delivery_location_latitude"],
                            location_info["Delivery_location_longitude"]], 
                   popup=location_info[["City", "Road_traffic_density"]]).add_to(map)
    
map

# 2 - Visão: entregadores

#### 2.1 - A menor e maior idade dos entregadores

In [52]:
print(f"Menor idade de entregador {df1.loc[:, 'Delivery_person_Age'].min()}.")
print(f"Maior idade de entregador {df1.loc[:, 'Delivery_person_Age'].max()}.")

Menor idade de entregador 20.
Maior idade de entregador 39.


#### 2.2 - A pior e a melhor condição de veículos

In [53]:
print(f"A pior condição de veículo: {df1.loc[:, 'Vehicle_condition'].min()}.")
print(f"A melhor condi;cão de veículo: {df1.loc[:, 'Vehicle_condition'].max()}.")

A pior condição de veículo: 0.
A melhor condi;cão de veículo: 2.


#### 2.3 - A avaliação média por entregador

In [54]:
df1.groupby("Delivery_person_ID").agg({"Delivery_person_Ratings":"mean"}).reset_index().rename(columns={"Delivery_person_Ratings":"Media_avaliação"})

,Delivery_person_ID,Media_avaliação
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000
...,...,...
1315,VADRES19DEL02,4.632727
1316,VADRES19DEL03,4.670270
1317,VADRES20DEL01,4.620370
1318,VADRES20DEL02,4.591111


#### 2.4 - A avaliação média e o desvio padrão por tipo de tráfego

In [55]:
delivery_mean_std = df1.groupby("Road_traffic_density").agg({"Delivery_person_Ratings":["mean", "std"]})
delivery_mean_std.columns = ["Delivery_mean", "Delivery_std"]
delivery_mean_std.reset_index(inplace=True)
delivery_mean_std

,Road_traffic_density,Delivery_mean,Delivery_std
0,High,4.652230,0.273044
1,Jam,4.594019,0.329778
2,Low,4.645011,0.338080
3,Medium,4.660138,0.274245


#### 2.5 - A avaliação média e o desvio padrão por condições climáticas

In [56]:
weather_mean_std = df1.groupby("Weatherconditions").agg({"Delivery_person_Ratings":["mean", "std"]})
weather_mean_std.columns = ["Delivery_mean", "Delivery_std"]
weather_mean_std.reset_index(inplace=True)
weather_mean_std

,Weatherconditions,Delivery_mean,Delivery_std
0,conditions Cloudy,4.651871,0.281197
1,conditions Fog,4.652965,0.275060
2,conditions Sandstorms,4.611748,0.310852
3,conditions Stormy,4.611819,0.313096
4,conditions Sunny,4.654868,0.396674
5,conditions Windy,4.616128,0.304565


#### 2.6 - Os 10 estregadores mais rápidos por cidade

In [57]:
mais_rapido_por_cidade = (
    (
        df1
        .groupby(["City", "Delivery_person_ID"])
        .agg({"Time_taken(min)":"mean"})
        .reset_index()
    )
    .groupby("City")
    .apply(lambda x: x.nsmallest(n=10, columns="Time_taken(min)"))
    .reset_index(drop=True)
)
mais_rapido_por_cidade

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,KNPRES03DEL02,15.750000
1,Metropolitian,ALHRES02DEL02,17.800000
2,Metropolitian,KNPRES01DEL01,19.125000
3,Metropolitian,KOLRES01DEL03,19.125000
4,Metropolitian,KOCRES02DEL02,19.250000
5,Metropolitian,KOCRES08DEL03,19.800000
6,Metropolitian,KOCRES16DEL03,20.000000
7,Metropolitian,KOCRES02DEL03,20.375000
8,Metropolitian,GOARES14DEL01,20.400000
9,Metropolitian,DEHRES20DEL03,20.428571


#### 2.7 - Os 10 entregadores mais lentos por cidade.

In [58]:
mais_lentos_por_cidade = (
    (
        df1
        .groupby(["City", "Delivery_person_ID"])
        .agg({"Time_taken(min)":"mean"})
        .reset_index()
    )
    .groupby("City")
    .apply(lambda x: x.nlargest(n=10, columns="Time_taken(min)"))
    .reset_index(drop=True)
)
mais_lentos_por_cidade

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AGRRES02DEL01,39.428571
1,Metropolitian,AURGRES11DEL03,38.500000
2,Metropolitian,KOLRES03DEL03,38.142857
3,Metropolitian,LUDHRES17DEL03,37.000000
4,Metropolitian,ALHRES18DEL02,36.666667
5,Metropolitian,GOARES08DEL03,36.000000
6,Metropolitian,ALHRES12DEL01,35.000000
7,Metropolitian,AGRRES13DEL02,34.700000
8,Metropolitian,KNPRES06DEL01,34.555556
9,Metropolitian,ALHRES010DEL01,34.500000


# 3 - Visão: Restaurante

#### 3.1 - A quantidade de entregadores únicos.

In [59]:
print(f"Quantidade de entregadores únicos: {df1['Delivery_person_ID'].nunique()}")

Quantidade de entregadores únicos: 1320


#### 3.2 - Distância média dos restaurantes e dos locais de entrega

In [66]:
from haversine import haversine, Unit


df1["distancia_restaurante"] = (
        df1
        .apply(lambda x: haversine((x["Restaurant_latitude"], x["Restaurant_longitude"]), 
                                (x["Delivery_location_latitude"], x["Delivery_location_longitude"]), 
                                    unit=Unit.KILOMETERS),
                                    axis=1)
    )
round(df1["distancia_restaurante"].mean(), 2)

26.91

#### 3.3 - O tempo médio e o desvio padrão de entrega por cidade

In [61]:
tempo_media_cidade = df1.groupby("City").agg({"Time_taken(min)":["mean", "std"]})
tempo_media_cidade.columns = ["tempo_medio", "desvio_padrao_tempo"]
tempo_media_cidade.reset_index(inplace=True)
tempo_media_cidade

,City,tempo_medio,desvio_padrao_tempo
0,Metropolitian,27.429860,9.134286
1,Semi-Urban,49.710526,2.724992
2,Urban,23.209495,8.859189


In [68]:
import plotly.graph_objects as go

df_aux = df1.groupby("City").agg({"Time_taken(min)":["mean", "std"]})
df_aux.columns = ["avg_time", "std_time"]

df_aux.reset_index(inplace=True)

fig = go.Figure()
fig.add_trace(go.Bar(name="Control",
                    x=df_aux["City"],
                    y=df_aux["avg_time"],
                    error_y=dict(type="data", array=df_aux["std_time"])))

fig.update_layout(barmode="group")

#### 3.4 - O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido

In [62]:
tempo_media_cidade_tipo_pedido = df1.groupby(["City", "Type_of_order"]).agg({"Time_taken(min)":["mean", "std"]})
tempo_media_cidade_tipo_pedido.columns = ["tempo_medio", "desvio_padrao_tempo"]
tempo_media_cidade_tipo_pedido.reset_index(inplace=True)
tempo_media_cidade_tipo_pedido

,City,Type_of_order,tempo_medio,desvio_padrao_tempo
0,Metropolitian,Buffet,27.301935,9.155644
1,Metropolitian,Drinks,27.324962,9.041698
2,Metropolitian,Meal,27.618877,9.215624
3,Metropolitian,Snack,27.467867,9.119682
4,Semi-Urban,Buffet,49.707317,2.731702
5,Semi-Urban,Drinks,49.625000,2.459347
6,Semi-Urban,Meal,50.300000,3.041665
7,Semi-Urban,Snack,49.408163,2.707385
8,Urban,Buffet,23.566337,9.060933
9,Urban,Drinks,23.308885,8.925721


#### 3.5 - O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego

In [63]:
tempo_media_cidade_tipo_trafego = df1.groupby(["City", "Road_traffic_density"]).agg({"Time_taken(min)":["mean", "std"]})
tempo_media_cidade_tipo_trafego.columns = ["tempo_medio", "desvio_padrao_tempo"]
tempo_media_cidade_tipo_trafego.reset_index(inplace=True)
tempo_media_cidade_tipo_trafego

,City,Road_traffic_density,tempo_medio,desvio_padrao_tempo
0,Metropolitian,High,28.145976,7.904494
1,Metropolitian,Jam,31.978193,9.477798
2,Metropolitian,Low,22.259874,6.795802
3,Metropolitian,Medium,27.728045,8.309769
4,Semi-Urban,High,50.125000,2.629956
5,Semi-Urban,Jam,49.841270,2.717095
6,Semi-Urban,Medium,47.400000,2.011080
7,Urban,High,24.305335,8.494842
8,Urban,Jam,27.989541,10.078679
9,Urban,Low,19.445964,6.319847


#### 3.6 - O tempo médio de entrega durante os Festivais

In [64]:
print(f"Tempo médio de entrega durante os Festivais: {df1[df1['Festival'] == 'Yes']['Time_taken(min)'].mean().round()} min")

Tempo médio de entrega durante os Festivais: 46.0 min
